In [18]:
from dataloader import *
        
if __name__ == '__main__':

    feats2level = {
        'jitter': 'utt',
        'shimmer': 'utt',
        'rp': 'utt',
        'f0': 'frame',
        'energy': 'frame'
    }

    np.set_printoptions(precision=2)
    features_to_load = ['jitter', 'shimmer', 'rp', 'f0', 'energy']
    # features_to_load = ['energy', 'rp']
    # for feat in allfeats:
    
    base_folder = '/data/storage025/Turntaking/wavs_single_channel_normalized_nosil'

    # Initialize the merged dataset
    merged_dataset = FeatureDataset(
        base_folder_path=base_folder,
        feature_names=features_to_load,
    )


Loading feature: jitter
Processing PictureNaming folder...
Found 1652 npy files
0 files with all 0 values
Processing EarlyLate folder...
Found 3971 npy files
nan in /data/storage025/Turntaking/wavs_single_channel_normalized_nosil/EarlyLate-features/jitter/subj-2120_27_E_kwaad_geloof.wav_1.wav_jitter.npy
0 files with all 0 values
Processing BoundaryTone folder...
Found 5026 npy files
0 files with all 0 values
Loaded 7918 samples for feature 'jitter'

Loading feature: shimmer
Processing PictureNaming folder...
Found 1652 npy files
0 files with all 0 values
Processing EarlyLate folder...
Found 3971 npy files
nan in /data/storage025/Turntaking/wavs_single_channel_normalized_nosil/EarlyLate-features/shimmer/subj-2120_27_E_kwaad_geloof.wav_1.wav_shimmer.npy
0 files with all 0 values
Processing BoundaryTone folder...
Found 5026 npy files
0 files with all 0 values
Loaded 7918 samples for feature 'shimmer'

Loading feature: rp
Processing PictureNaming folder...
Found 1652 npy files
0 files with

In [16]:

    # Perform train-test split
    train_dataset, test_dataset = train_test_split_by_subject(
        merged_dataset,
        feature_names=features_to_load,
        test_size=0.2,        # 20% for testing
        random_state=523       # For reproducibility
    )

    print(f"\nTraining Dataset: {len(train_dataset)} samples")
    print(f"Testing Dataset: {len(test_dataset)} samples")

    # Create DataLoaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=16,      # Adjust based on your memory constraints
        shuffle=True,       # Shuffle for training
        num_workers=4       # Adjust based on your CPU cores
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=16,      # Adjust as needed
        shuffle=False,      # No need to shuffle for testing
        num_workers=4       # Adjust based on your CPU cores
    )
    
    # Get NumPy arrays for scikit-learn (classification based on 'group_id')
    X_train, y_train = train_dataset.get_numpy(label_column='group_id')
    X_test, y_test = test_dataset.get_numpy(label_column='group_id')

    print("\nShape of X_train:", X_train.shape)
    print("Shape of y_train:", y_train.shape)
    print("Shape of X_test:", X_test.shape)
    print("Shape of y_test:", y_test.shape)

    print("\nFirst 5 training labels:", y_train[:5])
    print("First 5 testing labels:", y_test[:5])
    

    # # Example: Iterate over the training DataLoader
    # for batch_idx, batch in enumerate(train_loader):
    #     # Access features and metadata
    #     print(batch)
    #     print(len(batch))
    #     # ... and so on for other metadata and features

    #     # Example: Print batch information
    #     print(f"Train Batch {batch_idx + 1}:")
 

    #     # Break after the first batch for demonstration
    #     break



Total subjects: 52
Training subjects: 41
Testing subjects: 11
Total training samples: 6240
Total testing samples: 1578

Gender distribution in test set:
gender
V    0.520913
M    0.479087
Name: proportion, dtype: float64

Group distribution in test set:
group_id
21    0.616603
22    0.383397
Name: proportion, dtype: float64

Training Dataset: 6240 samples
Testing Dataset: 1578 samples

Shape of X_train: (6240, 3)
Shape of y_train: (6240,)
Shape of X_test: (1578, 3)
Shape of y_test: (1578,)

First 5 training labels: [1 1 0 1 0]
First 5 testing labels: [1 0 0 1 0]


In [17]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    roc_auc_score,
    roc_curve,
)

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(
    n_estimators=100,          # Number of trees
    max_depth=None,            # Maximum depth of each tree
    min_samples_split=2,       # Minimum samples to split a node
    min_samples_leaf=1,        # Minimum samples at a leaf node
    random_state=42,           # Seed for reproducibility
    class_weight='balanced'    # Adjust weights inversely proportional to class frequencies
)

# Train the model
rf_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_classifier.predict(X_test)
y_pred_proba = rf_classifier.predict_proba(X_test)[:, 1]

# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Generate Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Generate Classification Report
class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

# Calculate ROC-AUC
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC-AUC: {roc_auc:.4f}")

# Generate Confusion Matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# Calculate Sensitivity and Specificity
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")



Accuracy: 0.5602
Confusion Matrix:
[[724 249]
 [445 160]]
Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.74      0.68       973
           1       0.39      0.26      0.32       605

    accuracy                           0.56      1578
   macro avg       0.51      0.50      0.50      1578
weighted avg       0.53      0.56      0.54      1578

ROC-AUC: 0.4985
Sensitivity (Recall): 0.2645
Specificity: 0.7441
